# Fine-Tuning Qwen3 Models on Amazon SageMaker - Environment Preparation

This comprehensive notebook guides you through setting up a complete environment for fine-tuning Qwen3 language models on Amazon SageMaker. We'll cover everything from initial setup to data preparation, ensuring you have all the necessary components for successful model training.

## Learning Objectives

By the end of this notebook, you will understand how to:

- **Environment Setup**: Configure required packages, dependencies, and Docker settings for optimal performance
- **Model Management**: Download, prepare, and upload Qwen3 models and tokenizers from Hugging Face Hub
- **Data Preparation**: Structure and format training datasets for Chain-of-Thought reasoning tasks
- **Cloud Integration**: Upload prepared data and models to Amazon S3 for seamless SageMaker integration
- **Best Practices**: Implement efficient caching strategies and storage optimization techniques

## Prerequisites

- AWS Account with SageMaker access
- SageMaker Notebook Instance (ml.t3.medium or larger recommended)
- IAM role with appropriate SageMaker, S3, and ECR permissions
- Basic understanding of transformer models and fine-tuning concepts

**Important Note:** All notebooks in this series have been tested specifically on SageMaker Notebook instances. Local environments may require additional configuration.

## Package Installation and Docker Configuration

The first critical step involves setting up our development environment with the necessary machine learning libraries and optimizing Docker configuration for better performance during model training.

### What We're Installing

- **Core ML Libraries**: PyTorch, Transformers, Datasets for model handling
- **Fine-tuning Tools**: PEFT (Parameter-Efficient Fine-Tuning), TRL (Transformer Reinforcement Learning)
- **Optimization Libraries**: Accelerate for distributed training, BitsAndBytes for quantization
- **Cloud Integration**: SageMaker SDK for seamless AWS integration

### Docker Optimization

We'll configure Docker to use local storage with increased shared memory, which significantly improves performance during data loading and model training operations.

In [ ]:
# Installation control flag - set this based on your environment status
# True: Install all required packages (recommended for first run or clean environment)
# False: Skip installation if packages are already installed and up-to-date
install_needed = True
# install_needed = False  # Uncomment this line if packages are already installed

In [ ]:
%%bash
#!/bin/bash

# Docker configuration parameters for optimal performance
DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

# Check if Docker has already been configured with custom data-root
FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Docker configuration already optimized for SageMaker"
else
    echo "Configuring Docker for optimal performance..."
    
    # Stop Docker service for configuration changes
    sudo service docker stop
    
    echo "Adding data-root and default-shm-size=$MEMORY_SIZE to Docker configuration"
    
    # Backup existing Docker configuration
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    
    # Add custom data-root and shared memory size to Docker daemon configuration
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    
    # Migrate existing Docker data to new location
    sudo rsync -aP /var/lib/docker /home/ec2-user/SageMaker/.container
    
    # Restart Docker with new configuration
    sudo service docker start
    echo "Docker configuration complete and service restarted"
fi

# Optional: Install Docker Compose (uncomment if needed)
# sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
# sudo chmod +x /usr/local/bin/docker-compose

In [ ]:
# Import necessary modules for package installation and kernel management
import sys
import IPython

# Install required packages if needed
if install_needed:
    print("Installing dependencies and restarting kernel for clean environment...")
    
    # Upgrade pip to ensure compatibility with latest packages
    !{sys.executable} -m pip install --upgrade pip --quiet
    
    # Install comprehensive ML and fine-tuning package suite:
    # - sagemaker: AWS SageMaker SDK for training, deployment, and model management
    # - transformers: Hugging Face library for state-of-the-art transformer models
    # - datasets: Efficient data loading and processing for ML datasets
    # - peft: Parameter-Efficient Fine-Tuning techniques (LoRA, AdaLoRA, etc.)
    # - trl: Transformer Reinforcement Learning library for advanced training techniques
    # - accelerate: Distributed training and mixed precision support
    # - bitsandbytes: Quantization and memory optimization for large models
    !{sys.executable} -m pip install -U sagemaker transformers datasets peft trl accelerate bitsandbytes --quiet
    
    # Restart kernel to ensure all packages are properly loaded and avoid import conflicts
    IPython.Application.instance().kernel.do_shutdown(True)

## IAM Role Configuration for SageMaker

When running SageMaker in a local environment, you need access to an IAM Role with comprehensive permissions for SageMaker operations. This role should include:

- **SageMaker Full Access**: For training job creation and management
- **S3 Access**: For data and model artifact storage
- **ECR Access**: For custom container registry operations
- **CloudWatch Logs**: For monitoring and debugging training jobs

For detailed information about SageMaker IAM roles and required permissions, refer to the [AWS SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

## Core Library Imports and SageMaker Session Configuration

This section establishes our connection to AWS services and sets up the foundational components for our fine-tuning pipeline. We'll configure the SageMaker session, establish S3 bucket access, and verify our execution role permissions.

### Key Components

- **SageMaker Session**: Manages communication with AWS SageMaker services
- **S3 Integration**: Handles data storage and retrieval for training artifacts
- **IAM Role**: Provides necessary permissions for cross-service operations

In [ ]:
# Import essential libraries for AWS integration and file handling
import sagemaker  # AWS SageMaker SDK for ML operations
from pathlib import Path  # Modern Python path handling
from time import strftime  # Timestamp formatting for versioning

# Initialize SageMaker session - this handles all communication with AWS SageMaker
sagemaker_session = sagemaker.Session()

# Get the default S3 bucket for this SageMaker session
# This bucket will store our training data, model artifacts, and checkpoints
bucket = sagemaker_session.default_bucket()

# Retrieve the IAM execution role that SageMaker will use for training jobs
# This role must have permissions for S3, ECR, CloudWatch, and SageMaker operations
role = sagemaker.get_execution_role()

In [ ]:
# Verify SageMaker SDK version for compatibility
sagemaker.__version__

In [ ]:
import os

# Configure Hugging Face cache directories for persistent storage
# This ensures all downloaded models and datasets persist across notebook restarts
# and prevents unnecessary re-downloading of large model files

# Set cache directory for Hugging Face datasets - stores processed datasets locally
os.environ['HF_DATASETS_CACHE'] = '/home/ec2-user/SageMaker/.cache'

# Set general Hugging Face cache directory - stores various HF artifacts
os.environ['HF_CACHE_HOME'] = '/home/ec2-user/SageMaker/.cache'

# Set cache directory for Hugging Face Hub - stores downloaded model files
os.environ['HUGGINGFACE_HUB_CACHE'] = '/home/ec2-user/SageMaker/.cache'

# Additional cache directories (uncomment if experiencing cache-related issues)
# os.environ['TRANSFORMERS_HOME'] = '/home/ec2-user/SageMaker/.cache'
# os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/.cache'

## Model and Tokenizer Download and S3 Upload

This comprehensive section handles the complete lifecycle of model preparation for SageMaker training. We'll download the Qwen3 model and its associated tokenizer from Hugging Face Hub, save them locally for inspection and validation, and then upload them to S3 for use in distributed training environments.

### Process Overview

1. **Model Selection**: Choose the appropriate Qwen3 variant based on computational requirements
2. **Local Download**: Retrieve complete model repository including weights, configuration, and tokenizer
3. **Validation**: Verify model integrity and tokenizer functionality
4. **S3 Upload**: Prepare models for SageMaker training job access

### Why This Step is Essential

SageMaker training jobs run in isolated environments and need access to base model files through S3. Pre-downloading and uploading ensures faster training job startup and eliminates potential network issues during training initialization.

In [ ]:
# Import essential libraries for model handling and data processing
from transformers import (
    AutoModelForCausalLM,  # For loading causal language models (GPT-style)
    AutoTokenizer          # For loading and managing tokenizers
)
import torch                # PyTorch deep learning framework
from datasets import load_dataset  # Efficient dataset loading and processing
import huggingface_hub     # Direct Hub API access for model downloads
from trl import setup_chat_format  # Chat format setup utilities (if needed)

In [ ]:
# Define the target Qwen model for fine-tuning
# Qwen3-4B offers an excellent balance between performance and computational efficiency
# with 4 billion parameters - suitable for most fine-tuning scenarios

# Alternative model options (uncomment to use different variants):
# test_model_id = 'Qwen/Qwen2.5-3B-Instruct'  # Smaller instruction-tuned variant
# test_model_id = 'Qwen/Qwen3-7B'              # Larger variant for complex tasks

# Selected model for this tutorial - optimal for learning and experimentation
test_model_id = 'Qwen/Qwen3-4B'  # Base Qwen3 model with 4B parameters

In [ ]:
# Uncomment the following line if you need to authenticate with Hugging Face Hub
# This is required for accessing gated models or private repositories
# huggingface_hub.login()

In [ ]:
# Create a clean, filesystem-friendly directory name from the model ID
# This converts 'Qwen/Qwen3-4B' to 'qwen3-4b' for local storage organization
registered_model = test_model_id.split("/")[-1].lower().replace(".", "-")
print(f"Local model directory name: {registered_model}")

# Create the local directory structure to store the downloaded model
os.makedirs(registered_model, exist_ok=True)

# Download the complete model repository from Hugging Face Hub
# This includes model weights, configuration files, tokenizer files, and metadata
print(f"Downloading {test_model_id} from Hugging Face Hub...")
print("This may take several minutes depending on model size and network speed.")

huggingface_hub.snapshot_download(
    repo_id=test_model_id,      # The model repository identifier
    revision="main",            # Use the main branch (latest stable version)
    local_dir=registered_model  # Local directory to save all model files
)

print(f"Model download completed successfully to: {registered_model}/")

In [ ]:
# Load and verify the tokenizer for the Qwen3 model
# The tokenizer converts text to tokens and handles special tokens, padding, etc.
print("Loading and validating tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(test_model_id)

# Save the tokenizer to the local directory to ensure all components are available
# This creates a complete, self-contained model package
print("Saving tokenizer to local directory...")
tokenizer.save_pretrained(f'./{registered_model}')

print(f"Tokenizer saved successfully. Vocabulary size: {tokenizer.vocab_size}")

In [ ]:
# Upload the complete model package to S3 for SageMaker training access
# SageMaker training jobs require model files to be accessible via S3
print("Uploading model files to S3...")
print("This may take several minutes for large models.")

model_weight_path = sagemaker_session.upload_data(
    path=f'./{registered_model}',           # Local path containing all model files
    bucket=bucket,                          # Target S3 bucket
    key_prefix=f"checkpoints/{registered_model}"  # S3 key prefix (creates folder structure)
)

# Display the S3 URI where the model is now stored
print(f'✅ Model successfully uploaded to S3 path: {model_weight_path}')
print(f'This path will be used in training job configuration.')

## Dataset Selection and Preparation

For this tutorial, we're using a specialized Korean Chain-of-Thought reasoning dataset that will teach our model to provide step-by-step thinking before delivering final answers. This approach significantly improves the model's reasoning capabilities and response quality.

### Dataset Features

- **Chain-of-Thought Format**: Each example includes explicit reasoning steps
- **Diverse Topics**: Covers various domains and complexity levels
- **Structured Output**: Clear separation between thinking process and final answer

### Training Modes

We provide two training modes:
- **Quick Testing**: 200 samples + additional examples for rapid experimentation
- **Full Training**: Complete dataset for production-quality fine-tuning

In [ ]:
# Define the dataset for Chain-of-Thought reasoning fine-tuning
# This Korean dataset contains reasoning patterns that help models learn
# to provide systematic, step-by-step thinking before final answers
hkcode_dataset = "llami-team/Korean-OpenThoughts-114k-Normalized"

In [ ]:
# Configure training dataset size based on your requirements
# For experimentation and quick iterations, use the test subset
# For production fine-tuning, use the full dataset

test_yn = True   # Use subset: 200 samples + 9 additional examples for rapid testing
# test_yn = False  # Use full dataset: ~114k samples for comprehensive training

In [ ]:
# Determine dataset split based on training mode selection
# Testing mode: first 200 samples for quick experimentation
# Production mode: entire training set for comprehensive fine-tuning
split = "train[0:200]" if test_yn else "train"

# Load the Chain-of-Thought reasoning dataset from Hugging Face Hub
print(f"Loading dataset: {hkcode_dataset}")
print(f"Dataset split: {split}")
print("Loading dataset - this may take a moment...")

dataset = load_dataset(
    hkcode_dataset,        # Dataset repository name on Hugging Face Hub
    split=split,           # Which portion of the dataset to load
    trust_remote_code=True # Allow execution of custom dataset loading code
)

print(f"✅ Dataset loaded successfully. Total samples: {len(dataset)}")

## Additional Training Examples

To enhance the model's performance across diverse topics and improve its Chain-of-Thought reasoning capabilities, we're adding carefully crafted examples that cover various domains including tourism, culinary arts, philosophy, and general knowledge.

### Example Categories

- **Geographic and Cultural Information**: Detailed explanations about locations and cultural aspects
- **Culinary and Restaurant Knowledge**: Information about famous chefs and dining experiences
- **Classical Literature and Philosophy**: Educational content about traditional texts
- **Factual Information**: Current statistics and data-driven responses
- **Conversational AI Behavior**: Meta-questions about AI capabilities and limitations

Each example follows the structured format with question, reasoning process, and final response to maintain consistency with the main dataset.

In [ ]:
# Additional high-quality training examples to enhance model capabilities
# These examples demonstrate proper Chain-of-Thought reasoning across various domains
additional_items = [
    {
        "question": "What are the major tourist attractions and recommended routes in Seoul?",
        "reasoning": "Seoul is the capital of South Korea with numerous tourist attractions. To answer this question comprehensively, I should categorize Seoul's major tourist routes by theme. I can organize them into: historical/cultural routes, modern routes, nature routes, trendy routes, and traditional market routes. For each route, I should select about 3 representative locations and briefly explain their characteristics and highlights. Adding information about transportation methods and tips would also be useful.",
        "response": "Seoul offers diverse tourist attractions and captivating routes. Here are Seoul's representative tourist courses:\n\n## Historical and Cultural Route\n\n### Gyeongbokgung Palace - Bukchon Hanok Village - Insadong Course\n\n- **Gyeongbokgung Palace**: The first royal palace of the Joseon Dynasty, where you can admire magnificent architecture and beautiful gardens.\n- **Bukchon Hanok Village**: A well-preserved traditional village where you can enjoy hanbok experiences, traditional workshops, and galleries.\n- **Insadong**: The center of traditional culture with antique shops, galleries, and traditional tea houses.\n\n## Modern Seoul Route\n\n### Myeongdong - Namsan Seoul Tower - Dongdaemun Design Plaza (DDP) Course\n\n- **Myeongdong**: A paradise for shopping and food with various brand stores and street food.\n- **Namsan Seoul Tower**: Seoul's iconic landmark offering panoramic city views.\n- **DDP**: A complex cultural space famous for its unique architectural design.\n\n## Nature Route\n\n### Hangang Park - Yeouido - Seonyudo Park Course\n\n- **Hangang Park**: Perfect for cycling, picnicking, and riverside walks.\n- **Yeouido**: Financial district famous for cherry blossom festivals in spring.\n- **Seonyudo Park**: An ecological park created from a former water treatment plant with a unique atmosphere.\n\n## Trendy Seoul Route\n\n### Hongdae - Yeonnam-dong - Gyeongui Line Forest Trail Course\n\n- **Hongdae**: Youth culture center with various cafes, bars, clubs, and unique shops.\n- **Yeonnam-dong**: A hip neighborhood with charming cafes and restaurants.\n- **Gyeongui Line Forest Trail**: A park created from old railway tracks, perfect for walking.\n\n## Traditional Market Experience Route\n\n### Gwangjang Market - Dongdaemun Market - Namdaemun Market Course\n\n- **Gwangjang Market**: A historic market where you can taste various traditional foods.\n- **Dongdaemun Market**: A 24-hour fashion mecca.\n- **Namdaemun Market**: A large market where you can purchase various products at affordable prices.\n\nThese routes are designed to experience Seoul's diverse aspects. You'll be able to feel all of Seoul's charms including history, culture, modernity, nature, and shopping. Each route is easily accessible by public transportation, and using Seoul Bike (public bike rental) stations located throughout the city will make your travel even more convenient."
    },
    {
        "question": "How many Michelin 3-star restaurants are there worldwide?",
        "reasoning": "To answer this question, I need to provide the latest information based on Michelin Guide data about the number of 3-star restaurants worldwide. As of my knowledge cutoff, there were approximately 135 restaurants in 2022, but this number changes annually, so I should provide updated figures for 2024. I should also mention some representative 3-star restaurants from major countries to increase the credibility of my answer. I should include famous restaurants from countries with many Michelin 3-star establishments like France, Japan, USA, and Italy, and also mention the situation in Korea.",
        "response": "As of 2024, there are 145 Michelin 3-star restaurants worldwide. Here are representative 3-star restaurants by major countries:\n\n**France**\nFrance has the most Michelin 3-star restaurants in the world.\n- Restaurant Paul Bocuse (L'Auberge du Pont de Collonges): The world's oldest 3-star restaurant\n- Guy Savoy (Paris)\n- L'Ambroisie (Paris)\n\n**Japan**\nJapan has the second-highest number of 3-star restaurants after France.\n- Sukiyabashi Jiro (Tokyo)\n- Kanda (Tokyo)\n- Kikunoi Honten (Kyoto)\n\n**United States**\nThe US has approximately 12 three-star restaurants.\n- Masa (New York): Chef Takayama Masa's omakase restaurant\n- Alinea (Chicago): Restaurant offering unique olfactory experiences\n- The French Laundry (California)\n\n**Germany**\n- JAN (Munich): Notable restaurant that jumped directly from 0 to 3 stars in 2023\n\n**Spain**\n- Arzak (San Sebastian)\n- El Celler de Can Roca (Girona)\n\n**Italy**\n- Osteria Francescana (Modena)\n- Piazza Duomo (Alba)\n\n**Korea**\nKorea currently has no operating 3-star restaurants. 'Mosu', which maintained 3 stars until recently, has closed its operations."
    },
    {
        "question": "Tell me about famous chefs in Korea",
        "reasoning": "To answer this question, I should select famous Korean chefs and introduce their major achievements, restaurants, cooking styles, etc. It would be good to include a balanced mix of chefs who have received Michelin stars, chefs known to the public through TV programs, and chefs recognized internationally. For each chef, I should briefly introduce their representative restaurants, cooking philosophy, major awards, and mention their impact on the Korean culinary world. Chefs like Ahn Sung-jae, Yoo Hyun-soo, Cho Hee-sook, Kim Sun, and Choi Hyun-seok should be included.",
        "response": "Korea has several renowned chefs recognized both domestically and internationally. Let me introduce the major chefs:\n\n## Chef Ahn Sung-jae\n- Former owner chef of Michelin 3-star restaurant 'Mosu Seoul' (now closed).\n- Famous for innovative cuisine using Korean ingredients to create new dishes.\n- Pursues harmony between tradition and innovation, saying 'I added my own technique to respect for tradition.'\n\n## Chef Yoo Hyun-soo\n- Owner chef of restaurant 'Isipsajulgi' (Twenty-Four Seasons).\n- Known for contemplating different cooking methods while utilizing various Korean ingredients.\n\n## Chef Cho Hee-sook\n- Owner chef of 'Hansikgonggan', recipient of Michelin 1 star.\n- Famous for modern reinterpretation of traditional Korean cuisine.\n\n## Chef Kim Sun\n- Chef of restaurant 'Meta' in Singapore, recipient of Michelin 1 star.\n- Known for creating distinctive cuisine with Korean ingredients and sensibilities.\n\n## Chef Choi Hyun-seok\n- Owner chef of 'L'Amant Secret', became famous to the public through TV programs.\n\n## Chef Lim Ki-hak\n- Chef of restaurant 'Les Trois'.\n- Also known for providing sharp evaluations of other chefs' cuisine.\n\nIn addition, many other famous chefs including Kwon Woo-joong, Kim Dae-cheon, Kim Eun-hee, Park Se-min, Lee Jun, Lee Hyun-hee, and Jang Jin-mo are active in Korea's culinary world. They contribute to the development of Korean cuisine by presenting innovative dishes at their respective restaurants or communicating with the public through TV program appearances."
    },
    {
        "question": "Tell me about famous chefs in America",
        "reasoning": "To answer this question, I should select famous American chefs and introduce their major achievements, restaurants, cooking styles, and their impact on American culinary culture. It would be good to include a balanced mix from historically important chefs to modern famous chefs by era. I should introduce chefs who laid the foundation of American cuisine like Julia Child, James Beard, and Alice Waters, as well as chefs representing modern American cuisine like Thomas Keller, Anthony Bourdain, and Wolfgang Puck. For each chef, I should mention their representative restaurants, cooking philosophy, and their popular influence through TV programs.",
        "response": "America has many famous chefs who have greatly influenced culinary culture and industry. Here are the major American chefs:\n\n1. **Julia Child**\n- Pioneer who introduced French cuisine to America\n- Famous as host of TV cooking show 'The French Chef'\n\n2. **James Beard**\n- Called the 'Father of American Cuisine'\n- His legacy continues through the James Beard Foundation Awards\n\n3. **Alice Waters**\n- Pioneer of California cuisine and the farm-to-table movement\n- Owner chef of famous restaurant Chez Panisse\n\n4. **Thomas Keller**\n- Owner chef of Michelin 3-star restaurants The French Laundry and Per Se\n- Famous for sophisticated French-American cuisine\n\n5. **Anthony Bourdain**\n- Active as chef, author, and TV host\n- Gained great popularity with programs combining cooking and travel\n\n6. **Wolfgang Puck**\n- Pioneer of celebrity chefs\n- Operates upscale restaurant chains and various culinary businesses\n\n7. **Bobby Flay**\n- Popular cooking show host on Food Network\n- Known as a specialist in Southwestern cuisine\n\n8. **David Chang**\n- Founder of Momofuku restaurant group\n- Innovator of Asian-American fusion cuisine\n\nThese chefs have each developed American culinary culture in their own way and have greatly influenced the public through TV programs, books, and restaurants."
    },
    {
        "question": "What is the extent of your knowledge cutoff date?",
        "reasoning": "This question is about the knowledge cutoff date of the AI model. I need to accurately explain the last date of my training data and the actual knowledge cutoff date. I should also clarify the difference between the currently set date and the actual date of knowledge I possess. I should honestly answer so that users can understand the limitations that may arise when requesting the latest information.",
        "response": "My training data includes information up to April 2023. However, my precise knowledge cutoff date is September 2022. Therefore, information or events after that may be limited or inaccurate. While the current date is set to September 20, 2024, this is for conversational context and I don't actually possess information up to that date."
    },
    {
        "question": "Can you respond to inappropriate or sexual jokes?",
        "reasoning": "This question is about the AI model's content policy and ethical guidelines. I need to clearly explain my response policy regarding sexually explicit content or inappropriate jokes. I should politely explain to the user why I don't respond to such content and suggest healthy and beneficial conversation instead. It's important to help them understand that this is to maintain a safe and respectful conversational environment.",
        "response": "I don't respond to inappropriate jokes or sexually explicit content. Instead, I'd prefer to have healthy and beneficial conversations. If you'd like to continue our conversation on other topics, please let me know."
    },
    {
        "question": "Please explain the Analects of Confucius",
        "reasoning": "This question asks for an explanation of the Analects, a core Confucian text. I should systematically explain the author and compilation period, composition, main ideas, influence, and major passages of the Analects. I should emphasize the historical and philosophical importance of this book containing dialogues between Confucius and his disciples, and mention its impact on East Asian culture. It would be effective to explain the core values emphasized in the Analects such as ren (benevolence), li (ritual propriety), yi (righteousness), and xiao (filial piety), and provide a few representative passages as examples to aid understanding.",
        "response": "The Analects (論語, Lunyu) is a core Confucian classic containing the dialogues and teachings of Confucius and his disciples. Here are the main details:\n\n1. **Author and Compilation Period**\n- Compiled by Confucius's disciples and later scholars\n- Estimated to have been completed over about 300 years from the Spring and Autumn period to the early Han Dynasty\n\n2. **Composition**\n- Consists of 20 volumes and 492 chapters\n- Records Confucius's words and actions in dialogue format\n\n3. **Main Ideas**\n- Ren (仁): The fundamental virtue and highest moral quality of humanity\n- Li (禮): Standards of social order and morality\n- Yi (義): Moral standards for judging right and wrong\n- Xiao (孝): Respect and care for parents\n\n4. **Influence**\n- Foundational text of Confucian thought\n- Long used as a guide for education and politics in East Asian culture\n\n5. **Major Passages**\n- 'To learn and practice what you learn from time to time, is that not a pleasure?' (學而時習之 不亦說乎)\n- 'Virtue is not solitary; it always has neighbors' (德不孤 必有隣)\n\nThe Analects is a key text for understanding Confucian thought, providing deep insights into human nature, morality, politics, and education. It continues to be studied as an important classic in East Asian culture to this day."
    },
    {
        "question": "You are male. Nurses are generally female. Can we say that all nurses are female?",
        "reasoning": "This question deals with gender stereotypes and occupational bias. The question contains two incorrect premises: 1) the assumption that I am male and 2) the stereotype that nurses are generally female. I should provide accurate information while correcting these gender stereotypes. I need to clarify that there are no gender restrictions in the nursing profession and mention the proportion and historical contributions of male nurses. I should also explain why nurses of diverse genders are needed in patient care. The response should be polite yet educational and help eliminate gender stereotypes about professions.",
        "response": "We cannot say that all nurses are female. As a male nurse, let me tell you:\n\n1. There are no gender restrictions in the nursing profession. Men can certainly become nurses, and many male nurses are actually working in the field.\n\n2. Looking at statistics, the proportion of male nurses is steadily increasing. Currently, about 11% of all nurses in the United States are male.\n\n3. Male nurses possess the same level of professionalism and caring qualities as female nurses.\n\n4. In fact, a balanced mix of male and female nurses is needed to meet the diverse needs of patients.\n\n5. Historically, there have been men who made great contributions to the nursing field, such as Walt Whitman and Edward Ryan.\n\nTherefore, the statement 'all nurses are female' is not true and is merely a stereotype about nurses. It's more appropriate to view nurses as medical professionals with expertise and dedication, regardless of gender."
    }
]

# Add the additional examples to our main dataset
for item in additional_items:
    dataset = dataset.add_item(item)

print(f"✅ Dataset enhanced successfully. Total samples after adding examples: {len(dataset)}")

## Chain-of-Thought Prompt Template Design

This section defines our specialized prompt template for Chain-of-Thought reasoning. The template structures the training data to teach the model to provide explicit reasoning steps before delivering final answers, significantly improving response quality and transparency.

### Template Structure

- **Question Section**: Clear presentation of the user's query
- **Thinking Section**: Detailed step-by-step reasoning process
- **Final Answer Section**: Concise, actionable response based on the reasoning

### Key Benefits

- **Improved Accuracy**: Explicit reasoning reduces errors
- **Better Transparency**: Users can follow the model's thought process
- **Enhanced Problem-Solving**: Systematic approach to complex questions

In [ ]:
# Define the Chain-of-Thought prompt template for training
# This template teaches the model to provide systematic reasoning before final answers
train_prompt_style = """You are an AI Assistant with advanced knowledge in reasoning, analysis, and problem-solving.
Provide the most appropriate answer to the <question>. Before presenting your <final> answer, develop a step-by-step thought process (chain of thoughts) to perform logical and accurate analysis of the <question>.

<question>
{}
</question>

### Guidelines:
- Skip unnecessary greetings or preambles, and start directly with <response>
- Do not repeat the question and answer
- Write the step-by-step thought process in sufficient detail, but keep the final answer concise

### Response Format:
<think>
    ### THINKING
    {}
</think>
<final>
    ### FINAL-ANSWER
    {}
</final>
"""

## Data Processing and Formatting

The following section is based on the comprehensive guide [Fine-Tuning Qwen3: A Step-by-Step Guide](https://www.datacamp.com/tutorial/fine-tuning-qwen3), adapted for our specific Chain-of-Thought training requirements.

### Processing Pipeline

1. **Template Application**: Apply our CoT template to each training example
2. **Token Management**: Ensure proper EOS token handling for training stability
3. **Format Validation**: Verify correct structure for fine-tuning compatibility
4. **Quality Assurance**: Check data integrity and format consistency

In [ ]:
# Get the End-of-Sequence token from the tokenizer
# EOS token is crucial for proper training sequence termination
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN for proper training

def formatting_prompts_func(examples):
    """
    Format training examples using the Chain-of-Thought template.
    
    This function takes raw dataset examples and applies our specialized CoT template,
    ensuring proper structure for fine-tuning with reasoning capabilities.
    
    Args:
        examples: Batch of dataset examples containing question, reasoning, and response
        
    Returns:
        Dictionary with formatted text ready for training
    """
    inputs = examples["question"]     # User questions/queries
    complex_cots = examples["reasoning"]  # Step-by-step reasoning process
    outputs = examples["response"]    # Final answers/responses
    
    texts = []
    
    # Process each example in the batch
    for question, cot, response in zip(inputs, complex_cots, outputs):
        # Apply our Chain-of-Thought template to create training text
        text = train_prompt_style.format(question, cot, response)
        
        # Ensure proper sequence termination with EOS token
        if not text.endswith(tokenizer.eos_token):
            text += tokenizer.eos_token
            
        texts.append(text)
    
    return {"text": texts}

In [ ]:
# Apply the formatting function to transform our dataset
print("Applying Chain-of-Thought formatting to dataset...")

dataset = dataset.map(
    formatting_prompts_func,  # Our custom formatting function
    batched=True,            # Process multiple examples at once for efficiency
)

print("✅ Dataset formatting completed successfully.")
print("\n📝 Sample formatted training example:")
print("=" * 80)
print(dataset["text"][2])  # Display a sample formatted example
print("=" * 80)

## Dataset Upload to S3

The final step involves saving our processed dataset locally and uploading it to S3 for SageMaker training access. This ensures our training data is properly formatted and accessible during the distributed training process.

### Upload Process

1. **Local Save**: Create JSON file with formatted training examples
2. **S3 Upload**: Transfer to SageMaker-accessible storage
3. **Path Management**: Store S3 URIs for training job configuration
4. **Verification**: Confirm successful upload and accessibility

In [ ]:
# Create a clean dataset name from the Hugging Face dataset identifier
dataset_name = hkcode_dataset.split('/')[-1].lower()

# Define local path for saving the training dataset
local_training_input_path = f'{Path.cwd()}/dataset/train'

# Create directory structure if it doesn't exist
Path(local_training_input_path).mkdir(parents=True, exist_ok=True)

# Save the formatted dataset locally as JSON
print("Saving formatted dataset locally...")
dataset.to_json(
    f"{local_training_input_path}/train_dataset.json", 
    orient="records",     # JSON format with each example as a record
    force_ascii=False     # Preserve non-ASCII characters (important for multilingual data)
)

# Upload the training dataset to S3 for SageMaker access
print("Uploading training dataset to S3...")
training_input_path = sagemaker_session.upload_data(
    path=f"{local_training_input_path}/train_dataset.json",  # Local file path
    bucket=bucket,                                           # Target S3 bucket
    key_prefix=f"{dataset_name}/train"                      # S3 folder structure
)

# Display upload confirmation and paths
print("\n✅ Dataset upload completed successfully!")
print(f"📍 Training dataset uploaded to: {training_input_path}")
print(f"📁 Local dataset saved at: {local_training_input_path}/train_dataset.json")
print(f"📊 Total training examples: {len(dataset)}")

## SageMaker Role Information

For reference and troubleshooting purposes, we'll display the SageMaker execution role name. This information is useful when configuring IAM permissions or debugging access issues during training.

In [ ]:
# Extract and display the SageMaker execution role name for reference
from sagemaker import get_execution_role
sagemaker_role_name = get_execution_role().rsplit('/', 1)[-1]
print(f"🔐 SageMaker Execution Role Name: {sagemaker_role_name}")
print(f"📋 Full Role ARN: {get_execution_role()}")

## Parameter Storage and Session Management

To ensure continuity across notebook sessions and facilitate the next steps in our fine-tuning pipeline, we'll store all essential parameters using Jupyter's magic commands. This allows seamless transition to subsequent notebooks without manual parameter re-entry.

### Stored Parameters

- **Model Configuration**: Model ID and local directory name
- **Storage Paths**: S3 bucket, model weights, and training data locations
- **Local Paths**: Dataset storage locations for debugging and inspection

These parameters will be automatically available in subsequent notebooks for training job configuration and model deployment.

In [ ]:
# Store all essential parameters for use in subsequent notebooks
# This ensures seamless workflow continuity across the fine-tuning pipeline

%store test_model_id
# Hugging Face model identifier

%store bucket
# S3 bucket for storing artifacts

%store model_weight_path
# S3 path to uploaded model files

%store training_input_path
# S3 path to training dataset

%store local_training_input_path
# Local dataset storage path

%store registered_model
# Clean model directory name

# Display confirmation of stored parameters
print("✅ All parameters stored successfully for next notebook session:")
print("=" * 60)
print(f"🤖 Model ID: {test_model_id}")
print(f"🪣 S3 Bucket: {bucket}")
print(f"⚖️  Model Weights Path: {model_weight_path}")
print(f"📚 Training Data Path: {training_input_path}")
print(f"💾 Local Dataset Path: {local_training_input_path}")
print(f"📁 Registered Model Name: {registered_model}")
print("=" * 60)
print("🚀 Ready to proceed to the next notebook for training configuration!")

In [ ]:
# Retrieve stored parameters (use this in subsequent notebooks)
%store -r

## Summary and Next Steps

Congratulations! You have successfully completed the environment preparation phase for fine-tuning Qwen3 models on Amazon SageMaker. Here's what we accomplished:

### ✅ Completed Tasks

1. **Environment Setup**: Installed all required ML libraries and configured Docker for optimal performance
2. **Model Preparation**: Downloaded and uploaded Qwen3-4B model and tokenizer to S3
3. **Dataset Processing**: Formatted Chain-of-Thought reasoning dataset with custom templates
4. **Data Upload**: Prepared and uploaded training data to S3 for SageMaker access
5. **Configuration Storage**: Saved all parameters for seamless workflow continuation

### 🚀 Next Steps

You're now ready to proceed to the next phase:

- **Training Configuration**: Set up SageMaker training job with appropriate instance types and hyperparameters
- **Fine-Tuning Execution**: Launch the actual fine-tuning process with PEFT techniques
- **Model Evaluation**: Test and validate the fine-tuned model's performance
- **Deployment**: Deploy the trained model for inference

All necessary components are now in place for successful Qwen3 fine-tuning!